In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from time import sleep
import pandas as pd
import json
import re


class set_scraper:
    stock_code = ""

    def __init__(self, stock_code):
        # Init webdriver with headless
        self.options = Options()
        self.options.headless = True
        self.driver = webdriver.Chrome(options=self.options)

        # Init constructor for scarping
        self.stock_code = stock_code
        self.company_info = dict()
        self.stock_info = dict()
        self.financial_period_cols_header = []
        self.financial_statistics_cols_header = []
        self.factsheet_statistics_cols_header = []

        # Init page and row mapper
        self.set_url_prefix = "https://www.set.or.th/set"
        self.set_url_suffix = "&language=th&country=TH"
        self.company_info_xpath_row_no = {
            "industry_group": 2,
            "business_type": 3
        }
        self.page_mapper = {
            "company_info": self.set_url_prefix + "/companyprofile.do?symbol=" + self.stock_code + "&ssoPageId=4" + self.set_url_suffix,
            "company_highlight": self.set_url_prefix + "/companyhighlight.do?symbol=" + self.stock_code + "&ssoPageId=5" + self.set_url_suffix,
            "factsheet": self.set_url_prefix + "/factsheet.do?symbol=" + self.stock_code + "&ssoPageId=3" + self.set_url_suffix
        }

    def jumper(self, page_key):
        print('Jumping to page key: ', page_key, ' with url: ', self.page_mapper.get(page_key))
        self.driver.get(self.page_mapper.get(page_key))

    def go_to_company_highlights_page(self):
        self.jumper('company_highlight')

    def go_to_factsheet_page(self):
        self.jumper('factsheet')

    def get_company_info_xpath(self, field_key):
        element_text = self.driver.find_element_by_xpath("/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-reponsive']/table[@class='table']/tbody/tr[3]/td/div[@class='row']/div[@class='col-xs-12 col-md-7']/div[@class='row'][" + str(self.company_info_xpath_row_no.get(field_key)) + "]/div[@class='col-xs-9 col-md-5']").text
        return element_text

    # Get company industry group and business type from SET website
    # Then save it into company_info variable which is already declared in constructor
    def get_company_info(self):
        self.jumper('company_info')

        for item in ['industry_group', 'business_type']:
            temp_value = self.get_company_info_xpath(item)
            self.company_info[item] = temp_value

    # Get financial period columns from SET website
    # Then append it into financial_period_cols_header variable in constructor
    def get_financial_period_column_header(self):
        df_stock = self.driver.find_elements_by_xpath(
            "/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/thead[1]/tr/th")

        for index, column in enumerate(df_stock):
            column_dict = dict()
            if (column.text.startswith("งบปี")):
                column_dict['index'] = index + 1
                column_dict['text'] = column.text.replace("\n", " - ")
                column_dict['year'] = re.findall("\d{4}", column.text)[0]
                if not column_dict['year'] in fipc_years:
                    fipc_years.add(column_dict['year'])
                self.financial_period_cols_header.append(column_dict)

    # Get financial statistics columns from SET website
    # Then append it into financial_statistics_cols_header variable in constructor
    def get_financial_statistics_column_header(self):
        df_stock = self.driver.find_elements_by_xpath(
            "/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/thead[2]/tr/th")

        for index, column in enumerate(df_stock):
            column_dict = dict()
            date_pattern = re.compile("^\d{1,2}\/\d{1,2}\/\d{4}$")
            if (date_pattern.match(column.text)):
                column_dict['index'] = index + 1
                column_dict['text'] = column.text
                column_dict['year'] = re.findall("\d{4}", column.text)[0]
                if not column_dict['year'] in fisc_years:
                    fisc_years.add(column_dict['year'])
                self.financial_statistics_cols_header.append(column_dict)

    # Get factsheet statistics column from SET website
    # Then append it into factsheet_statistics_cols_header variable in constructor
    def get_factsheet_statistics_column_header(self):
        column_list = self.driver.find_elements_by_xpath(
            "/html[@class='no-js']/body/table/tbody/tr[3]/td/table[@class='table-factsheet-padding3'][2]/tbody/tr[4]/td[2]/table[@class='table-factsheet-padding0'][1]/tbody/tr[2]/td[@class='factsheet-head']")

        for index, column in enumerate(column_list):
            column_dict = dict()
            if not column.text.startswith("ข้อมูลสถิติ") and index + 1 == 2:
                column_dict['index'] = index + 1
                column_dict['text'] = column.text.replace("\n", " - ")
                column_dict['year'] = re.findall("\d{4}", column.text)[0]
                if not column_dict['year'] in fasc_years:
                    fasc_years.add(column_dict['year'])
                self.factsheet_statistics_cols_header.append(column_dict)

    # Get net profit value from SET website by:
    # - Iterating over financial_period_cols_header variable to find which column index to get data from
    # - After that, trim founded text and push it into profit_list variable in constructor
    def get_net_profit(self):
        profit_list = []
        for index, column in enumerate(self.financial_period_cols_header):
            profit = self.driver.find_element_by_xpath(
                "/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/tbody[1]/tr[7]/td[" + str(column.get("index")) + "]").text
            profit_list.append(profit.strip())
        self.stock_info['profit_list'] = profit_list

    def get_pe_value(self):
        pe_list = []
        for index, column in enumerate(self.financial_statistics_cols_header):
            pe = self.driver.find_element_by_xpath(
                "/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/tbody[2]/tr[4]/td[" + str(column.get("index")) + "]").text
            pe_list.append(pe.strip())
        self.stock_info['pe_list'] = pe_list

    def get_bv_value(self):
        bv_list = []
        for index, column in enumerate(self.financial_statistics_cols_header):
            bv = self.driver.find_element_by_xpath(
                "/ html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/tbody[2]/tr[6]/td[" + str(column.get("index")) + "]").text
            bv_list.append(bv.strip())
        self.stock_info['bv_list'] = bv_list

    def get_p_bv_value(self):
        p_bv_list = []
        for index, column in enumerate(self.financial_statistics_cols_header):
            p_bv = self.driver.find_element_by_xpath(
                "/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/tbody[2]/tr[5]/td[" + str(column.get("index")) + "]").text
            p_bv_list.append(p_bv.strip())
        self.stock_info['p_bv_list'] = p_bv_list

    def get_roa_value(self):
        roa_list = []
        for index, column in enumerate(self.financial_period_cols_header):
            roa = self.driver.find_element_by_xpath(
                "/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/tbody[1]/tr[10]/td[" + str(column.get("index"))+ "]").text
            roa_list.append(roa.strip())
        self.stock_info['roa_list'] = roa_list

    def get_roe_value(self):
        roe_list = []
        for index, column in enumerate(self.financial_period_cols_header):
            roe = self.driver.find_element_by_xpath(
                "/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/tbody[1]/tr[11]/td[" + str(column.get("index")) + "]").text
            roe_list.append(roe.strip())
        self.stock_info['roe_list'] = roe_list

    def get_eps_value(self):
        eps_list = []
        for index, column in enumerate(self.financial_period_cols_header):
            eps = self.driver.find_element_by_xpath(
                "/html[@class='no-js']/body/div[@class='container']/div[@class='row sidebar-body-content']/div[@id='body-content']/div[@class='row']/div[@id='maincontent']/div[@class='row']/div[@class='table-responsive']/table[@class='table table-hover table-info']/tbody[1]/tr[8]/td[" + str(column.get("index")) + "]").text
            eps_list.append(eps.strip())
        self.stock_info['eps_list'] = eps_list

    def get_beta_value(self):
        beta_list = []
        for index, column in enumerate(self.factsheet_statistics_cols_header):
            beta = self.driver.find_element_by_xpath(
                "/html[@class='no-js']/body/table/tbody/tr[3]/td/table[@class='table-factsheet-padding3'][2]/tbody/tr[4]/td[2]/table[@class='table-factsheet-padding0'][1]/tbody/tr[11]/td[@class='factsheet'][" + str(column.get("index")) + "]").text
            beta_list.append(beta.strip())
        self.stock_info['beta_list'] = beta_list

    def get_benefit_value(self):
        self.stock_info['benefit_value'] = "https://www.set.or.th/set/companyrights.do?symbol=" + self.stock_code + "&ssoPageId=7&language=th&country=TH"

    def get_value_from_page(self): 
        print('get value from page')

    def get_company_highlights(self):
        self.go_to_company_highlights_page()
        self.get_financial_period_column_header()
        self.get_financial_statistics_column_header()
        self.get_net_profit()
        self.get_pe_value()
        self.get_bv_value()
        self.get_p_bv_value()
        self.get_roa_value()
        self.get_roe_value()
        self.get_eps_value()
        self.go_to_factsheet_page()
        self.get_factsheet_statistics_column_header()
        self.get_beta_value()
        self.get_benefit_value()

    def retrieve_stock_info(self):
        self.get_company_info()
        # self.get_company_highlights()


# Save list of years
fipc_years = set([])
fisc_years = set([])
fasc_years = set([])

df_stock = dict()
df_stock = {
    'industry_group': [],
    'business_type': [],
    'stock_name': [],
}
unsorted_stock_keys = {}

# Get list of stock by reading it from file or define in variable
# stock_list = open("stockList.txt", "r")
stock_list = ['MINT', 'CPALL']

# Start iteration over list of stocks
for index, stock in enumerate(stock_list):
    scraper = set_scraper(stock)
    scraper.retrieve_stock_info()

    df_stock['industry_group'].append(scraper.company_info.get('industry_group'))
    df_stock['business_type'].append(scraper.company_info.get('business_type'))
    df_stock['stock_name'].append(stock.replace("\n", ""))

    for fipc_index, year in enumerate(fipc_years):
        if not '1. กำไรสุทธิ ' + year in unsorted_stock_keys:
            unsorted_stock_keys['1. กำไรสุทธิ ' + year] = []
        if not '2. ROA ' + year in unsorted_stock_keys:
            unsorted_stock_keys['2. ROA ' + year] = []
        if not '3. ROE ' + year in unsorted_stock_keys:
            unsorted_stock_keys['3. ROE ' + year] = []
        if not '4. กำไรต่อหุ้น ' + year in unsorted_stock_keys:
            unsorted_stock_keys['4. กำไรต่อหุ้น ' + year] = []

        unsorted_stock_keys['1. กำไรสุทธิ ' + year].append(scraper.stock_info.get('profit_list')[fipc_index])
        unsorted_stock_keys['2. ROA ' + year].append(scraper.stock_info.get('roa_list')[fipc_index])
        unsorted_stock_keys['3. ROE ' + year].append(scraper.stock_info.get('roe_list')[fipc_index])
        unsorted_stock_keys['4. กำไรต่อหุ้น ' + year].append(scraper.stock_info.get('eps_list')[fipc_index])

    for fisc_index, year in enumerate(fisc_years):
        if not '5. P/E ' + year in unsorted_stock_keys:
            unsorted_stock_keys['5. P/E ' + year] = []
        if not '6. BV ' + year in unsorted_stock_keys:
            unsorted_stock_keys['6. BV ' + year] = []
        if not '7. P/BV ' + year in unsorted_stock_keys:
            unsorted_stock_keys['7. P/BV ' + year] = []

        unsorted_stock_keys['5. P/E ' + year].append(scraper.stock_info.get('pe_list')[fisc_index])
        unsorted_stock_keys['6. BV ' + year].append(scraper.stock_info.get('bv_list')[fisc_index])
        unsorted_stock_keys['7. P/BV ' + year].append(scraper.stock_info.get('p_bv_list')[fisc_index])

    for fasc_index, year in enumerate(fasc_years):
        if not '8. Beta ' + year in unsorted_stock_keys:
            unsorted_stock_keys['8. Beta ' + year] = []

        unsorted_stock_keys['8. Beta ' + year].append(scraper.stock_info.get('beta_list')[fasc_index])

    if not '9. Benefit' in unsorted_stock_keys:
        unsorted_stock_keys['9. Benefit'] = []
    
    unsorted_stock_keys['9. Benefit'].append(scraper.stock_info.get('benefit_value'))

    # If iteration comes to the end, perform sorting stock keys and merge it into data variables
    if index == len(stock_list) - 1:
        sorted_stock_keys = {}

        for index_key, key in enumerate(sorted(unsorted_stock_keys.keys())):
            sorted_stock_keys[key] = unsorted_stock_keys[key]

        df_stock = {**df_stock, **sorted_stock_keys}


df = pd.DataFrame(data=df_stock)
df

Jumping to page key:  company_info  with url:  https://www.set.or.th/set/companyprofile.do?symbol=MINT&ssoPageId=4&language=th&country=TH
company_info {'industry_group': 'เกษตรและอุตสาหกรรมอาหาร', 'business_type': 'อาหารและเครื่องดื่ม'}
Jumping to page key:  company_info  with url:  https://www.set.or.th/set/companyprofile.do?symbol=CPALL&ssoPageId=4&language=th&country=TH
company_info {'industry_group': 'บริการ', 'business_type': 'พาณิชย์'}


,industry_group,business_type,stock_name,9. Benefit
0,เกษตรและอุตสาหกรรมอาหาร,อาหารและเครื่องดื่ม,MINT,None
1,บริการ,พาณิชย์,CPALL,None
